In [1]:
import pandas as pd
import re

In [2]:
cd ~/Desktop/DataSamples/R&D/ViCell

C:\Users\agilman\Desktop\DataSamples\R&D\ViCell


In [3]:
filename = "BSR5.xlsx"
df = pd.read_excel(filename)

# ViCell Data Cleaning 

In [4]:
#Getting list of strings from rows 3 and 4. These are the column headers that ViCell split into 2 rows.

clms_list1 = []
clms_list2 = []
for i in df.loc[3,:]:
    clms_list1.append(i)
for i in df.loc[4,:]:
    clms_list2.append(i)
    
#Combining the strings from rows 3 and 4 to be the new column name 

combined = []
for i,j in zip(clms_list1, clms_list2):
    if (type(i) == str) and (type(j) == str):    
        combined.append(i+j)
    else:
        combined.append(i)

df.columns = combined #setting the columns to combined strings from rows 3 and 4/

#dropping first 6 rows and resetting index

df.drop(df.index[0:6], inplace = True)
df.reset_index(inplace = True, drop = True)


#dropping duplicates (just in case), becomes more important after mergin multiple vicell files
df.drop_duplicates(subset = "Sample date/time", inplace = True) #dropping duplicate rows


##### NEED VERIFICATION FUNCTION TO MAKE SURE "Sample id" == "File name"
df["Sample ID"] = df["Sample ID"].str.slice(0,5) #shortening Sample ID colum to first 4 characters 
df["File name"] = df["File name"].str.slice(0,5) #shortening "File name" column


#isolating columns of interest, converting datatpes

df = df.loc[:,['Sample ID','Sample date/time','Viability(%)','Viable cells/ml (x10^6)']].copy()
df = df.apply(pd.to_numeric, errors = "ignore")
df['Sample date/time'] = pd.to_datetime(df['Sample date/time'])

#Sorting data by date/time
df.sort_values(by ="Sample date/time", inplace = True) #sorting chronologically 

#Renaming columns to prep for merge 
df.rename(columns = {"Sample date/time": "Vicell date/time"}, inplace = True)

#Unique ViCell Sample IDs
print("Accpeted unique sample IDs: " + str(df["Sample ID"].unique()))


# Importing FLEX data

In [6]:
cd ~/Desktop/DataSamples/R&D/Flex

C:\Users\agilman\Desktop\DataSamples\R&D\Flex


In [7]:
df_flex = pd.read_csv("SampleResults2019-11-21_143948.csv")

In [8]:
# Selecting wanted and unwanted columns
flex_columns = ['Sample ID', 'Date & Time', 'Gln', 'Glu', 'Gluc', 'Lac', 'NH4+', 'Na+', 'K+', 'Ca++', 'pH', 'PO2',
                'PCO2', 'O2 Saturation', 'Osm', 'Vessel Temperature (°C)','Chemistry Dilution Ratio','HCO3',]
flex_rejected_columns =  list(set(list(df_flex.columns)).difference(flex_columns)) #just in case want to check rejected clms

#trashing all columns except desired ones 
df_flex = df_flex[flex_columns].copy()

#Parsing wanted and unwanted samples
a = re.compile("[Rr][0-9][0-9]") #expression to capture, letter r or R, followed by 2 numbers, followed by anything else
df_flex = df_flex[df_flex["Sample ID"].str.match(a)].copy()
df_flex_reject = df_flex[~df_flex["Sample ID"].str.match(a)].copy() # ~ is the opposite of, or True when False

#printing accepted and rejected sample ID's
print("Accpeted unique sample IDs: " + str(df_flex["Sample ID"].unique()))
print("Rejected unique sample IDs: " + str(df_flex_reject["Sample ID"].unique()))


Accpeted unique sample IDs: ['R0015' 'R0018' 'R0017' 'R0016' 'r15 GLUC' 'r16 GLUC']
Rejected unique sample IDs: []


In [9]:
#function to change Sample Id's:
def rename_sample_id(dict_change, df):
    
    """
    Allows user to rename mislabled sample IDs. 
    Changing lists of mislabaled sample IDs to correct value (dictionary key)
    
    INPUTS:
    dict_change -  dictionary: values are lists of possible wrong Sample ID's, any matches will change to the key.
    df - dataframe containing column "Sample ID". Values in this column will be changed. 
    """
    for key, value in dict_change.items():
        id_match = df["Sample ID"].isin(dict_change[key])
        index = df[id_match].index
        df.loc[index,"Sample ID"] = key
        
    return df


In [10]:
#creating a dictionary to change all in list into key
dict_change = {"R0015 Gluc":["r15 GLUC"],
              "R0016 Gluc":["r16 GLUC"]}

rename_sample_id(dict_change, df_flex)

print("Accpeted unique sample IDs: " + str(df_flex["Sample ID"].unique()))
print("Rejected unique sample IDs: " + str(df_flex_reject["Sample ID"].unique()))


#Changing to datetime dtypes 
df_flex['Date & Time'] = pd.to_datetime(df_flex['Date & Time'])

#Sorting data by date/time
df_flex.sort_values(by ='Date & Time', inplace = True) #sorting chronologically 

#Renaming columns to make FLEX specific columns
df_flex.rename(columns = {'Date & Time': "Flex date/time"},
                        inplace = True)

Accpeted unique sample IDs: ['R0015' 'R0018' 'R0017' 'R0016' 'R0015 Gluc' 'R0016 Gluc']
Rejected unique sample IDs: []


# Playing with merging/joinng

In [15]:
df_vicell = df
print("vicell: " + str(df_vicell.shape))
print("flex: " + str(df_flex.shape))

vicell: (58, 5)
flex: (67, 19)


In [16]:
#Creating a scafold "rank" column in df_flex for later outer join
df_flex['Rank'] = df_flex['Flex date/time'].rank()

#left join on Vicell dataframe. merge_asof allows merging by closest values. will need outer join to get remainng right df data.

merged = pd.merge_asof(df_vicell, df_flex,
             left_on = "Vicell date/time",
             right_on = "Flex date/time",
             direction = "nearest",
             by = "Sample ID",
             tolerance = pd.Timedelta("30 minutes"))



In [17]:
merged.rename(columns = {"Sample ID":"ViCell Sample ID",
                        "Runtime": "Vicell Runtime"}, inplace = True)

In [18]:
#trmming down to only viCell columns from first merge, will use rank to do an outer merge with flex data
columns_to_keep = ['ViCell Sample ID', 'Vicell date/time', "Vicell Runtime", 'Viability(%)', 'Viable cells/ml (x10^6)', 'Rank']
merged = merged[columns_to_keep].copy()
merged = merged.merge(df_flex, on = "Rank", how = "outer").copy()


In [35]:
merged.rename(columns = {"Sample ID":"Flex Sample ID"}, inplace = True)

In [52]:
merged = merged.assign(Sample_ID = lambda x: x.filter(like = "Sample ID").bfill(1).iloc[:,0]).copy()
merged = merged.assign(datetime = lambda x: x.filter(like = "date/time").bfill(1).iloc[:,0]).copy()
merged = merged.assign(Runtime = lambda x: x.filter(like = "Runtime").bfill(1).iloc[:,0]).copy()


In [55]:
ordered_clms = ['ViCell Sample ID', 'Flex Sample ID',
                'Vicell date/time', 'Flex date/time',
                
                'Vicell Runtime', 'Flex Runtime',
               'Sample_ID', 'datetime', 'Runtime',
                
               'Viable cells/ml (x10^6)', 'Viability(%)', 'Rank',
               'Flex date/time', 'Gln', 'Glu', 'Gluc', 'Lac', 'NH4+', 'Na+', 'K+',
               'Ca++', 'pH', 'PO2', 'PCO2', 'O2 Saturation', 'Osm',
               'Vessel Temperature (°C)', 'Chemistry Dilution Ratio', 'HCO3',]

In [56]:
data = merged[ordered_clms].copy()

In [81]:
data.sort_values(by = ["datetime"], inplace = True)

In [82]:
data[data["Sample_ID"].str.slice(0,5) == "R0015"]

,ViCell Sample ID,Flex Sample ID,Vicell date/time,Flex date/time,Vicell Runtime,Flex Runtime,Sample_ID,datetime,Runtime,Viable cells/ml (x10^6),...,K+,Ca++,pH,PO2,PCO2,O2 Saturation,Osm,Vessel Temperature (°C),Chemistry Dilution Ratio,HCO3
0,R0015,R0015,2019-11-01 08:34:23,2019-11-01 08:31:29,0.000000,0.000000,R0015,2019-11-01 08:34:23,0.000000,0.504,...,12.90,0.17,7.166,150.8,65.1,100.0,359.0,37.0,1:1,23.7
1,R0015,R0015,2019-11-02 09:31:01,2019-11-02 09:27:44,1.039329,1.039062,R0015,2019-11-02 09:31:01,1.039329,1.150,...,12.72,0.16,7.108,108.2,50.6,72.6,359.0,37.0,1:1,16.2
2,R0015,R0015,2019-11-03 08:45:41,2019-11-03 08:38:57,2.007847,2.005185,R0015,2019-11-03 08:45:41,2.007847,2.990,...,12.89,0.16,7.024,129.9,12.6,87.2,346.0,37.0,1:1,3.3
58,NaN,R0015,NaT,2019-11-04 09:08:14,NaN,3.025521,R0015,2019-11-04 09:08:14,3.025521,NaN,...,12.79,0.16,6.740,85.3,11.7,57.2,350.0,37.0,1:1,1.6
3,R0015,R0015,2019-11-05 09:03:49,2019-11-05 09:00:48,4.020440,4.020359,R0015,2019-11-05 09:03:49,4.020440,11.400,...,12.51,0.18,6.907,88.1,34.2,59.2,362.0,37.0,1:1,6.9
4,R0015,R0015,2019-11-06 10:16:47,2019-11-06 10:10:47,5.071111,5.068958,R0015,2019-11-06 10:16:47,5.071111,25.300,...,12.54,0.19,6.763,44.4,36.4,29.8,353.0,37.0,1:1,5.3
8,R0015,R0015,2019-11-07 09:12:53,2019-11-07 09:08:17,6.026736,6.025556,R0015,2019-11-07 09:12:53,6.026736,27.500,...,12.77,0.22,6.772,49.0,38.9,32.9,352.0,37.0,1:2,5.7
16,R0015,R0015,2019-11-08 09:04:58,2019-11-08 09:00:28,7.021238,7.020127,R0015,2019-11-08 09:04:58,7.021238,57.900,...,12.31,0.17,6.847,28.1,42.9,15.3,345.0,34.0,1:1,7.1
20,R0015,R0015,2019-11-09 12:44:09,2019-11-09 12:38:50,8.173449,8.171771,R0015,2019-11-09 12:44:09,8.173449,79.500,...,12.13,0.16,6.871,12.9,37.8,7.0,307.0,34.0,1:1,6.6
24,R0015,R0015,2019-11-10 12:57:32,2019-11-10 12:53:41,9.182743,9.182083,R0015,2019-11-10 12:57:32,9.182743,96.800,...,13.13,0.14,6.916,0.0,43.6,0.0,293.0,34.0,1:1,8.5


# Calculating Runtime for ViCell and Flex dataframes

In [13]:
#calculating Vicell runtime 
df["Runtime"] = 0 #empty column with zeros 

for key, grp in df.groupby(["Sample ID"]):
    
    time_delta = grp["Vicell date/time"].diff() #time difference 
    time_delta = time_delta.dt.total_seconds() / (24 * 60 * 60) #converting to float
    added_time = time_delta.cumsum() #added time 
    added_time.iloc[0] = 0 #setting the first value to zero instead of Nan 

    ind = added_time.index

    df.loc[ind,"Runtime"] = added_time

In [14]:
#  calculating Flex Runtime 

df_flex["Flex Runtime"] = 0 #empty column with zeros 


for key, grp in df_flex.groupby(["Sample ID"]):
    
    time_delta = grp["Flex date/time"].diff() #time difference 
    time_delta = time_delta.dt.total_seconds() / (24 * 60 * 60) #converting to float
    added_time = time_delta.cumsum() #added time 
    added_time.iloc[0] = 0 #setting the first value to zero instead of Nan 

    ind = added_time.index

    df_flex.loc[ind,"Flex Runtime"] = added_time